In [1]:
# read nc output from WAOM 10km run

import xarray as xr
import pandas as p
import numpy as np
import numpy.ma as ma
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl
# mpl.use('Agg')
mpl.use('TKAgg')
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.ticker import NullFormatter
from matplotlib.colors import LinearSegmentedColormap   # for custom colormaps
from matplotlib import colors
import matplotlib.path as mpath

from datetime import datetime, timedelta

from netCDF4 import Dataset
from netCDF4 import num2date, date2num

import gsw
import pyresample

# read grid file for lon/lat coordinates
dg = xr.open_dataset("/scratch/project_2000339/boeiradi/waom10_frc/waom10extend_grd.nc")
lat_rho_10km= dg.variables["lat_rho"]
lon_rho_10km = dg.variables["lon_rho"]
lat_u_10km= dg.variables["lat_u"]
lon_u_10km = dg.variables["lon_u"]
lat_v_10km= dg.variables["lat_v"]
lon_v_10km = dg.variables["lon_v"]
cor_10km = dg.variables["f"]
pm_10km = dg.variables["pm"]
pn_10km = dg.variables["pn"]
zice_10km = dg.variables["zice"]
dg.close()
print('Print lon/lat_rho shapes',lon_rho_10km.shape, lat_rho_10km.shape)

dg4 = xr.open_dataset("/scratch/project_2000789/boeiradi/waom4_frc/waom4extend_grd.nc")
lat_rho_4km = dg4.variables["lat_rho"]
lon_rho_4km = dg4.variables["lon_rho"]
lat_u_4km= dg4.variables["lat_u"]
lon_u_4km = dg4.variables["lon_u"]
lat_v_4km= dg4.variables["lat_v"]
lon_v_4km = dg4.variables["lon_v"]
cor_4km = dg4.variables["f"]
pm_4km = dg4.variables["pm"]
pn_4km = dg4.variables["pn"]
zice_4km = dg4.variables["zice"]
dg4.close()

dg2 = xr.open_dataset("/scratch/project_2000339/boeiradi/waom2_frc/waom2extend_grd.nc")
lat_rho_2km = dg2.variables["lat_rho"]
lon_rho_2km = dg2.variables["lon_rho"]
lat_u_2km= dg2.variables["lat_u"]
lon_u_2km = dg2.variables["lon_u"]
lat_v_2km= dg2.variables["lat_v"]
lon_v_2km = dg2.variables["lon_v"]
cor_2km = dg2.variables["f"]
pm_2km = dg2.variables["pm"]
pn_2km = dg2.variables["pn"]
zice_2km = dg2.variables["zice"]
dg2.close()

Print lon/lat_rho shapes (560, 630) (560, 630)


In [5]:
# read ROMS 2km:
def read_roms_ts_2km(exp_path):
    for yr in ['09','10']:
        ds = xr.open_dataset(exp_path + 'ocean_avg_00' + yr + '.nc')
        print('size temp and time length: ', ds.temp.shape, len(ds.salt.isel(xi_rho=20, eta_rho=100, s_rho=0)))
        temp_tmp = ds.variables["temp"]
        salt_tmp = ds.variables["salt"]
#         zeta_tmp = ds.variables["zeta"]
        temp_tmp_ann = np.nanmean(temp_tmp, axis=0)
        salt_tmp_ann = np.nanmean(salt_tmp, axis=0)
        print('size temp_tmp_ann = ', temp_tmp_ann.shape)
        del temp_tmp, salt_tmp

        # calculate geostrophic velocities from ssh:
#         g = 9.81
#         dzetady = np.diff(zeta_tmp, axis=1)
#         dzetadx = np.diff(zeta_tmp, axis=2)
#         mmax = len(zeta_tmp)
#         dzeta_dy = np.empty((dzetady[:,:,0:-1].shape))
#         dzeta_dx = np.empty((dzetadx[:,0:-1,:].shape))
#         for mm in np.arange(0,mmax,1):
#             dzeta_dy[mm,:,:] = dzetady[mm,:,0:-1]*pn_2km[0:-1,0:-1]
#             dzeta_dx[mm,:,:]= dzetadx[mm,0:-1,:]*pm_2km[0:-1,0:-1]
#         ugeo = np.empty((dzeta_dy.shape))
#         vgeo = np.empty((dzeta_dx.shape))
#         for mm in np.arange(0,mmax,1):
#             ugeo[mm,:,:] = -(g/cor_2km[0:-1,0:-1])*dzeta_dy[mm,:,:]
#             vgeo[mm,:,:] = (g/cor_2km[0:-1,0:-1])*dzeta_dx[mm,:,:]
#         ugeo_ann_tmp = np.nanmean(ugeo, axis=0)
#         vgeo_ann_tmp = np.nanmean(vgeo, axis=0)
        #ugeo_anom_tmp = ugeo - ugeo_ann
        #vgeo_anom_tmp = vgeo - vgeo_ann
#         V_ann_tmp = np.sqrt(ugeo_ann_tmp*ugeo_ann_tmp + vgeo_ann_tmp*vgeo_ann_tmp)
#         MKE_tmp = np.divide(V_ann_tmp*V_ann_tmp,2)
#         print("Size MKE_tmp", MKE_tmp.shape)

#         ugeo_anom_tmp = ugeo - ugeo_ann_tmp
#         vgeo_anom_tmp = vgeo - vgeo_ann_tmp
#         V_anom_tmp = np.sqrt(ugeo_anom_tmp*ugeo_anom_tmp + vgeo_anom_tmp*vgeo_anom_tmp)
#         V_anom_ann = np.nanmean(V_anom_tmp, axis=0)
#         EKE_tmp = np.divide(V_anom_ann*V_anom_ann,2)
#         print("Size EKE_tmp", EKE_tmp.shape)

        ds = ds.set_coords(['Cs_r', 'Cs_w', 'hc', 'h', 'Vtransform'])

        if ds.Vtransform == 1:
            Zo_rho = ds.hc * (ds.s_rho - ds.Cs_r) + ds.Cs_r * ds.h
            z_rho_tmp = Zo_rho + ds.zeta * (1 + Zo_rho/ds.h)
            print("Vtransform=1")
        elif ds.Vtransform == 2:
            Zo_rho = (ds.hc * ds.s_rho + ds.Cs_r * ds.h) / (ds.hc + ds.h)
            z_rho_tmp = ds.zeta + (ds.zeta + ds.h) * Zo_rho
            print("Vtransform=2")
        z_rho_tmp_ann = np.nanmean(z_rho_tmp, axis=0)

#         Zo_w = (ds.hc * ds.s_w + ds.Cs_w * ds.h) / (ds.hc + ds.h)
#         z_w_tmp = ds.zeta + (ds.zeta + ds.h) * Zo_w
#         z_w_tmp_ann = np.nanmean(z_w_tmp, axis=0)

        # concantenate annual averaged temp/salt
        if yr == '09':
            temp_ann = np.expand_dims(temp_tmp_ann, axis=0)
            salt_ann = np.expand_dims(salt_tmp_ann, axis=0)
#             z_w_ann = np.expand_dims(z_w_tmp_ann, axis=0)
            z_rho_ann = np.expand_dims(z_rho_tmp_ann, axis=0)
#             MKE_ann = np.expand_dims(MKE_tmp, axis=0)
#             EKE_ann = np.expand_dims(EKE_tmp, axis=0)
        elif yr == '10':
            temp_ann = np.squeeze(np.nanmean(np.stack((temp_ann,np.expand_dims(temp_tmp_ann, axis=0)), axis=0), axis=0))
            salt_ann = np.squeeze(np.nanmean(np.stack((salt_ann,np.expand_dims(salt_tmp_ann, axis=0)), axis=0), axis=0))
#             z_w_ann = np.squeeze(np.nanmean(np.stack((z_w_ann,np.expand_dims(z_w_tmp_ann, axis=0)), axis=0), axis=0))
            z_rho_ann = np.squeeze(np.nanmean(np.stack((z_rho_ann,np.expand_dims(z_rho_tmp_ann, axis=0)), axis=0), axis=0))
#             MKE_ann = np.squeeze(np.nanmean(np.stack((MKE_ann,np.expand_dims(MKE_tmp, axis=0)), axis=0), axis=0))
#             EKE_ann = np.squeeze(np.nanmean(np.stack((EKE_ann,np.expand_dims(EKE_tmp, axis=0)), axis=0), axis=0))

        print('Annual temp and annual tmp temp sizes = ', temp_ann.shape, temp_tmp_ann.shape)

    return temp_ann, salt_ann#, MKE_ann, EKE_ann

In [6]:
path_ECCO2_2km = '/scratch/project_2000339/boeiradi/waom2extend_shflim_S_0.25Q/output_01-05yr/'
temp_ann_2km, salt_ann_2km = read_roms_ts_2km(path_ECCO2_2km)

size temp and time length:  (6, 31, 2800, 3150) 6


/scratch/project_2000339/boeiradi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
/scratch/project_2000339/boeiradi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


size temp_tmp_ann =  (31, 2800, 3150)
Vtransform=2


/scratch/project_2000339/boeiradi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice


Annual temp and annual tmp temp sizes =  (1, 31, 2800, 3150) (31, 2800, 3150)
size temp and time length:  (6, 31, 2800, 3150) 6


/scratch/project_2000339/boeiradi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
/scratch/project_2000339/boeiradi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


size temp_tmp_ann =  (31, 2800, 3150)
Vtransform=2


/scratch/project_2000339/boeiradi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice
/scratch/project_2000339/boeiradi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: Mean of empty slice
/scratch/project_2000339/boeiradi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: Mean of empty slice


Annual temp and annual tmp temp sizes =  (31, 2800, 3150) (31, 2800, 3150)


/scratch/project_2000339/boeiradi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: RuntimeWarning: Mean of empty slice


In [42]:
mask_zice_10km = ma.masked_where(zice_10km < 0, np.ones(zice_10km.shape))

## ==========================================
# interpolate WAOM4 and WAOM2 field to the WAOM10 grid for maps of difference:

w10_def = pyresample.geometry.SwathDefinition(lons=lon_rho_10km,lats=lat_rho_10km)
# w4_def = pyresample.geometry.SwathDefinition(lons=lon_rho_4km,lats=lat_rho_4km)
w2_def = pyresample.geometry.SwathDefinition(lons=lon_rho_2km,lats=lat_rho_2km)

wf = lambda r: 1/r

# bot_temp_ann_4km_interp = pyresample.kd_tree.resample_custom(w4_def,temp_ann_4km[0,:,:],w10_def,\
#                                          radius_of_influence=30000,neighbours=4,weight_funcs=wf)
bot_temp_ann_2km_interp = pyresample.kd_tree.resample_custom(w2_def,temp_ann_2km[0,:,:],w10_def,\
                                         radius_of_influence=5000,neighbours=1,weight_funcs=wf)


In [43]:
print(bot_temp_ann_2km_interp[0:50,0:50])

[[1.00070834 0.96313667 0.92828965 ... 1.41794586 1.40514874 1.37355828]
 [0.92756414 0.92480862 0.92282379 ... 1.39532804 1.37724495 1.37413144]
 [0.92240942 0.91760564 0.91564852 ... 1.37622416 1.36431789 1.37593853]
 ...
 [0.70442212 0.71080148 0.77163166 ... 0.67037189 0.67956412 0.67438895]
 [0.68837643 0.83016819 0.99723238 ... 0.6678955  0.73345476 0.55569047]
 [0.71251082 0.73652446 0.87587214 ... 0.65880847 0.7327379  0.49401188]]


In [ ]:
plt.pcolormesh(bot_temp_ann_2km_interp)
#plt.pcolormesh(temp_ann_2km[0,0:400,0:400])
plt.colorbar()
plt.show()